# Heart Segmentation Project

## Introduction
This project focuses on segmenting different parts of the heart using deep learning techniques. We will use a dataset of medical images and train a model to accurately segment regions of interest.

### Objectives
- Segment the right ventricle (RV)
- Segment the myocardium (MYO)
- Segment the left ventricle (LV)

## Dependencies Import and Exploration
This section covers the initial steps of importing every dependency for data manipulation, transformation, visualization, model handling, and other utilities. It also includes a command to mount Google Drive for accessing files in a Google Colab environment.

1. `gc` and `gc.collect()`: Imports the garbage collection module and calls the `collect` method to free up memory.
2. `random`: Provides functions for generating random numbers and making random selections.
3. `torchvision.transforms as t`: Imports the `transforms` module from `torchvision`, which provides image transformation functions.
4. `IPython.display import Image as IPImage`: Imports the `Image` class from `IPython.display` for displaying images in Jupyter notebooks.
5. `torch`: The main library for PyTorch, used for building and training neural networks.
6. `sys`: Provides access to system-specific parameters and functions.
7. `os`: Provides functions for interacting with the operating system.
8. `pandas as pd`: Imports pandas, a data manipulation and analysis library.
9. `matplotlib.pyplot as plt`: Imports the plotting library for creating static, animated, and interactive visualizations.
10. `numpy as np`: Imports NumPy, a library for numerical operations on arrays.
11. `torch.nn as nn`: Imports the neural network module from PyTorch.
12. `nibabel as nib`: Imports nibabel for reading and writing medical image formats.
13. `torch.nn.functional as F`: Imports functional operations for neural networks from PyTorch.
14. `pickle`: Provides functions for serializing and deserializing Python objects.
15. `torch.utils.data import Dataset, DataLoader, random_split`: Imports data handling classes from PyTorch.
16. `PIL import Image`: Imports the Python Imaging Library (PIL) for image processing.
17. `imageio`: Provides functions for reading and writing images.
18. `time`: Provides functions for time-related operations.
19. `torchvision.models as models`: Imports pre-trained models from `torchvision`.
20. `cv2`: Imports OpenCV for computer vision tasks.
21. `statistics import mode`: Imports the mode function for statistical operations.
22. `google.colab import drive` and `drive.mount('/content/gdrive')`: Imports the Google Drive module from Google Colab and mounts the Google Drive for accessing files.

In [ ]:
!pip install nibabel

In [ ]:
import gc
gc.collect()
import random
from torchvision.transforms import v2 as t
from IPython.display import Image as IPImage
import random
import torch
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import nibabel as nib
import torch.nn.functional as F
from torchvision.transforms import v2 as t
import random
import pickle
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import imageio
import time
from IPython.display import Image as IPImage
import gc
import torchvision.models as models
import cv2
from statistics import mode
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## import numpy as np
sys.path.append('/content/gdrive/MyDrive/finalproject-ruangguru')
import functs as m
from loss_function import Adaptive_tvMF_DiceLoss
from unet_model import U_Net
# device = device = xm.xla_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.device_count())
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

cuda 1


## Model Selection

In this project, we utilize two different neural network architectures for heart segmentation: DenseNet-121 and MobileNetV2. These models have been chosen due to their balance between accuracy and computational efficiency.

### DenseNet-121
> DenseNet-121 is a densely connected convolutional network architecture that improves the flow of information and gradients throughout the network, making it easier to train. It is known for achieving high performance on image classification tasks.

### MobileNetV2
> MobileNetV2 is a lightweight convolutional neural network designed for mobile and edge devices. Despite its smaller size and lower computational cost, it can achieve competitive accuracy. This model is less explored for the given dataset, hence its inclusion in this project aims to evaluate its performance in heart MRI segmentation.

## Model Loading

> Here, we load the pre-trained weights for both DenseNet-121 and MobileNetV2 models. These weights are fine-tuned on our specific dataset to perform the task of heart segmentation.

In [ ]:
data = m.load_train_data('/content/gdrive/MyDrive/finalproject-ruangguru/result1/')
data_mobile = m.load_train_data('/content/gdrive/MyDrive/finalproject-ruangguru/result-mobilenet/results/')

In [ ]:
data_test = m.ACDC_2D_Dataset(
    "/content/gdrive/MyDrive/finalproject-ruangguru/test.csv",
    "/content/gdrive/MyDrive/finalproject-ruangguru/ACDC",
    joint_transform = None,
    image_transform = None,
    train=False
)

## Optimizer and Loss Function

We use the Adam optimizer with a learning rate of 0.0005 and a weight decay of 0.0001 for regularization. The loss function is customized to handle the imbalances in the segmentation labels, with equal weighting for each class (left ventricle, myocardium, and ventricle).

In [ ]:
model = m.TrainedAutoencoder('densenet121',4)
model.load_state_dict(data['model_state_dict'])

model_mobile = m.TrainedAutoencoder('mobilenet_v2',4)
model_mobile.load_state_dict(data_mobile['model_state_dict'])

optim = torch.optim.Adam(model.parameters(), lr= 0.0005, weight_decay = 0.0001)
loss = m.CustomLoss(weights = [1.0, 1.0, 1.0])
metric = [m.ClassDice(channel = 1), m.ClassDice(channel = 2), m.ClassDice(channel=3)]

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 116MB/s]
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 97.3MB/s]


## Model Testing

The test function evaluates the performance of both DenseNet-121 and MobileNetV2 models on the test dataset. The metrics used for evaluation include class-specific Dice scores, which measure the overlap between the predicted and ground truth segmentations.

In [ ]:
loss, metric = m.test(DATASET = data_test,
                      MODEL = model,
                      MODEL2 = model_mobile,
                      LOSS = loss,
                      METRICS = metric,
                      SAVE_LOC = '/content/gdrive/MyDrive/finalproject-ruangguru/result1-test2')

In [ ]:
from google.colab import runtime
runtime.unassign()